In [1]:
import os
import keras
import scipy.io as sio
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2


Using TensorFlow backend.


## DEFINE MODEL

In [2]:
def classifier_model():
    """Build the classifier
    :returns: Classifier model
    :rtype: keras.Model
    """
    model = Sequential()
    model.add(Dense(512, input_dim=4096, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(32, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.6))
    model.add(Dense(1, kernel_initializer='glorot_normal',
                    kernel_regularizer=l2(0.001), activation='sigmoid'))
    return model

In [3]:
def conv_dict(dict2):
    """Prepare the dictionary of weights to be loaded by the network
    :param dict2: Dictionary to format
    :returns: The dictionary properly formatted
    :rtype: dict
    """
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

In [4]:
def load_weights(model, weights_file):
    """Loads the pretrained weights into the network architecture
    :param model: keras model of the network
    :param weights_file: Path to the weights file
    :returns: The input model with the weights properly loaded
    :rtype: keras.model
    """
    dict2 = sio.loadmat(weights_file)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

In [8]:
# build classifier and load pretrained weights
def create_classifier_model():
    model = classifier_model()
    model = load_weights(model, './weights_L1L2.mat')
    return model

In [9]:
model = create_classifier_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                16416     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 2,114,113
Trainable params: 2,114,113
Non-trainable params: 0
_________________________________________________________________


In [11]:
import keras.optimizers
from keras.models import model_from_json
import numpy as np
import keras.backend as K


from datetime import datetime

def save_model(model, json_path, weight_path):
    json_string = model.to_json()
    open(json_path, 'w').write(json_string)
    dict = {}
    i = 0
    for layer in model.layers:
        weights = layer.get_weights()
        my_list = np.zeros(len(weights), dtype=np.object)
        my_list[:] = weights
        dict[str(i)] = my_list
        i += 1
    sio.savemat(weight_path, dict)

def load_model(json_path):
    model = model_from_json(open(json_path).read())
    return model

def load_batch_train(normal_path, normal_list, abnormal_path, abnormal_list):

    batchsize=60
    n_exp = int(batchsize/2)

    num_normal = len(normal_list)
    num_abnormal = len(abnormal_list)

    abnor_list_idx = np.random.permutation(num_abnormal)
    abnor_list = abnor_list_idx[:n_exp]
    norm_list_idx = np.random.permutation(num_normal)
    norm_list = norm_list_idx[:n_exp]

    abnormal_feats = []
    for video_idx in abnor_list:
        video_path = os.path.join(abnormal_path, abnormal_list[video_idx])
        with open(video_path, "rb") as f:
            feats = np.load(f)
        abnormal_feats.append(feats)

    normal_feats = []
    for video_idx in norm_list:
        video_path = os.path.join(normal_path, normal_list[video_idx])
        with open(video_path, "rb") as f:
            feats = np.load(f)
        normal_feats.append(feats)


    all_feats = np.vstack((*abnormal_feats, *normal_feats))
    all_labels = np.zeros(32*batchsize, dtype='uint8')

    all_labels[:32*n_exp] = 1

    return  all_feats, all_labels


def custom_objective(y_true, y_pred):

    y_true = K.reshape(y_true, [-1])
    y_pred = K.reshape(y_pred, [-1])
    n_seg = 32
    nvid = 60
    n_exp = int(nvid / 2)

    max_scores_list = []
    z_scores_list = []
    temporal_constrains_list = []
    sparsity_constrains_list = []

    for i in range(0, n_exp, 1):

        video_predictions = y_pred[i*n_seg:(i+1)*n_seg]

        max_scores_list.append(K.max(video_predictions))
        temporal_constrains_list.append(
            K.sum(K.pow(video_predictions[1:] - video_predictions[:-1], 2))
        )
        sparsity_constrains_list.append(K.sum(video_predictions))

    for j in range(n_exp, 2*n_exp, 1):

        video_predictions = y_pred[j*n_seg:(j+1)*n_seg]
        max_scores_list.append(K.max(video_predictions))

    max_scores = K.stack(max_scores_list)
    temporal_constrains = K.stack(temporal_constrains_list)
    sparsity_constrains = K.stack(sparsity_constrains_list)

    for ii in range(0, n_exp, 1):
        max_z = K.maximum(1 - max_scores[:n_exp] + max_scores[n_exp+ii], 0)
        z_scores_list.append(K.sum(max_z))

    z_scores = K.stack(z_scores_list)
    z = K.mean(z_scores)

    return z + \
        0.00008*K.sum(temporal_constrains) + \
        0.00008*K.sum(sparsity_constrains)

output_dir = "/home/jupyter/project/"
normal_dir = "/home/jupyter/project/processed_normal_train_features"
abnormal_dir = "/home/jupyter/project/processed_abnormal_train_features"

normal_list = os.listdir(normal_dir)
normal_list.sort()
abnormal_list = os.listdir(abnormal_dir)
abnormal_list.sort()

weights_path = output_dir + 'weights_new.mat'

model_path = output_dir + 'model.json'

#Create Full connected Model
model = classifier_model()

adagrad = keras.optimizers.Adagrad(lr=0.001, epsilon=1e-08)
model.compile(loss=custom_objective, optimizer=adagrad)

if not os.path.exists(output_dir):
       os.makedirs(output_dir)

loss_graph =[]
num_iters = 20000
total_iterations = 0
batchsize=60
time_before = datetime.now()


for it_num in range(num_iters):
    inputs, targets = load_batch_train(
        normal_dir, normal_list, abnormal_dir, abnormal_list
    )
    batch_loss = model.train_on_batch(inputs, targets)
    loss_graph = np.hstack((loss_graph, batch_loss))
    total_iterations += 1
    if total_iterations % 20 == 0:
        print ("Iteration={} took: {}, loss: {}".format(
            total_iterations, datetime.now() - time_before, batch_loss)
        )

print("Train Successful - Model saved")
save_model(model, model_path, weights_path)


Iteration=20 took: 0:00:15.344571, loss: 30.762760162353516
Iteration=40 took: 0:00:21.867407, loss: 30.48362159729004
Iteration=60 took: 0:00:28.518407, loss: 30.23193359375
Iteration=80 took: 0:00:35.112095, loss: 29.88795280456543
Iteration=100 took: 0:00:41.823448, loss: 29.361705780029297
Iteration=120 took: 0:00:48.188143, loss: 29.60045051574707
Iteration=140 took: 0:00:54.680323, loss: 28.552034378051758
Iteration=160 took: 0:01:01.355970, loss: 28.53481101989746
Iteration=180 took: 0:01:07.887586, loss: 28.373279571533203
Iteration=200 took: 0:01:14.380443, loss: 27.12131118774414
Iteration=220 took: 0:01:20.870370, loss: 26.861955642700195
Iteration=240 took: 0:01:26.915887, loss: 26.70330810546875
Iteration=260 took: 0:01:33.410127, loss: 26.57275390625
Iteration=280 took: 0:01:39.675758, loss: 26.969945907592773
Iteration=300 took: 0:01:46.108856, loss: 25.599271774291992
Iteration=320 took: 0:01:52.531627, loss: 26.429174423217773
Iteration=340 took: 0:01:58.756344, loss: 